In [1]:

from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import math
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import app, flags
from easydict import EasyDict
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D, Dense, Flatten, Conv2D, MaxPool2D

from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt

from cleverhans.future.tf2.attacks import projected_gradient_descent, fast_gradient_method

import foolbox as fb
import eagerpy as ep
from foolbox import TensorFlowModel, accuracy, samples
import foolbox.attacks as fa
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [129]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False)
        self.pruned_w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='ones',
                                 trainable=False)
        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        self.pruned_w = tf.multiply(self.w, self.mask)
        #print('layer inputy', inputs.shape)
        x = tf.matmul(inputs, self.pruned_w)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')


class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        #print('NN call',inputs.shape)
        x = tf.keras.layers.Flatten()(inputs)
        x = self.dense1(x)
        
        x = self.dense2(x)
        #print(x.shape)
        return self.dense3(x)
    


In [33]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255


In [34]:
def ld_mnist():
    """Load training and test data."""

    def convert_types(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        return image, label

    dataset, info = tfds.load('mnist', 
                              data_dir='gs://tfds-data/datasets', 
                              with_info=True,
                              as_supervised=True)
    mnist_train, mnist_test = dataset['train'], dataset['test']
    mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(128)
    mnist_test = mnist_test.map(convert_types).batch(128)
    return EasyDict(train=mnist_train, test=mnist_test)
data = ld_mnist()

In [156]:
def initialize_model():
    model = LeNet300_100()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )
    return model

def train_model(model):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=data.train,
              #batch_size=64,
              epochs=5000,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )


In [168]:
pruning_ratios=[0,0.5,.8,.9,.95,.99]
for pruning_ratio in pruning_ratios:
    pruned_model = prune(model, pruning_ratio)
    train_model(pruned_model)
    attack(model)

yoyoyo <class 'numpy.ndarray'>
yoyoyo <class 'numpy.ndarray'>
yoyoyo <class 'numpy.ndarray'>
Epoch 1/5000
    469/Unknown - 11s 24ms/step - loss: 1.4667 - accuracy: 0.9946- 11s 24ms/step - loss: 1.4667 - accuracy: 0.Epoch 1/5000
469/469 [==============================] - 12s 25ms/step - loss: 1.4667 - accuracy: 0.9946 - val_loss: 1.4805 - val_accuracy: 0.9804
Epoch 2/5000
469/469 [==============================] - 11s 23ms/step - loss: 1.4664 - accuracy: 0.9950 - val_loss: 1.4805 - val_accuracy: 0.9809
Epoch 3/5000
469/469 [==============================] - 10s 21ms/step - loss: 1.4661 - accuracy: 0.9952 - val_loss: 1.4792 - val_accuracy: 0.9824
Epoch 4/5000
469/469 [==============================] - 11s 22ms/step - loss: 1.4662 - accuracy: 0.9952 - val_loss: 1.4810 - val_accuracy: 0.9799
Epoch 5/5000
469/469 [==============================] - 10s 22ms/step - loss: 1.4665 - accuracy: 0.9949 - val_loss: 1.4804 - val_accuracy: 0.9816
Epoch 6/5000
469/469 [==============================] 

Epoch 8/5000
469/469 [==============================] - 10s 22ms/step - loss: 1.4651 - accuracy: 0.9966 - val_loss: 1.4803 - val_accuracy: 0.9818
Epoch 9/5000
469/469 [==============================] - 10s 22ms/step - loss: 1.4649 - accuracy: 0.9966 - val_loss: 1.4806 - val_accuracy: 0.9811
Epoch 10/5000
469/469 [==============================] - 11s 23ms/step - loss: 1.4648 - accuracy: 0.9967 - val_loss: 1.4801 - val_accuracy: 0.9811
Epoch 11/5000
469/469 [==============================] - 11s 24ms/step - loss: 1.4646 - accuracy: 0.9968 - val_loss: 1.4802 - val_accuracy: 0.9808
Epoch 12/5000
469/469 [==============================] - 10s 21ms/step - loss: 1.4645 - accuracy: 0.9969 - val_loss: 1.4809 - val_accuracy: 0.9804
Epoch 13/5000
469/469 [==============================] - 10s 22ms/step - loss: 1.4644 - accuracy: 0.9969 - val_loss: 1.4801 - val_accuracy: 0.9812
Epoch 14/5000
469/469 [==============================] - 10s 20ms/step - loss: 1.4643 - accuracy: 0.9970 - val_loss: 1.4

469/469 [==============================] - 10s 20ms/step - loss: 2.1770 - accuracy: 0.3654 - val_loss: 2.1766 - val_accuracy: 0.3705
Epoch 29/5000
469/469 [==============================] - 10s 21ms/step - loss: 2.1769 - accuracy: 0.3656 - val_loss: 2.1764 - val_accuracy: 0.3709
Epoch 30/5000
469/469 [==============================] - 10s 21ms/step - loss: 2.1767 - accuracy: 0.3664 - val_loss: 2.1764 - val_accuracy: 0.3701
Epoch 31/5000
469/469 [==============================] - 10s 21ms/step - loss: 2.1766 - accuracy: 0.3663 - val_loss: 2.1762 - val_accuracy: 0.3705
Epoch 32/5000
469/469 [==============================] - 10s 21ms/step - loss: 2.1765 - accuracy: 0.3663 - val_loss: 2.1761 - val_accuracy: 0.3704
Epoch 33/5000
469/469 [==============================] - 10s 21ms/step - loss: 2.1764 - accuracy: 0.3671 - val_loss: 2.1761 - val_accuracy: 0.3709
Epoch 34/5000
469/469 [==============================] - 10s 21ms/step - loss: 2.1762 - accuracy: 0.3672 - val_loss: 2.1760 - val_ac

In [163]:
def attack(model):
    fmodel = fb.models.TensorFlowModel(model, bounds=(0,1))
    images_from_ds, labels_from_ds = get_mnist_data()
    attacks = [
        fa.FGSM(),
        fa.LinfPGD(),
    ]
    epsilons = [
        0.0,
        0.001,
        0.01,
        0.1,
        .2,
        .3,
        .4,
        .5,
        .6,
        .7,
        .8,
        .9,
        1.0,
    ]
    attack_success = np.zeros((len(attacks), len(epsilons), len(images_from_ds)), dtype=np.bool)
    for i, attack in enumerate(attacks):
        _, _, success = attack(fmodel, images_from_ds, labels_from_ds, epsilons=epsilons)
        assert success.shape == (len(epsilons), len(images_from_ds))
        success_ = success.numpy()
        assert success_.dtype == np.bool
        attack_success[i] = success_
        print(attack)
        print("  ", 1.0 - success_.mean(axis=-1).round(2))
    
    

In [131]:
fmodel = fb.models.TensorFlowModel(model, bounds=(0,1))

In [160]:
def get_mnist_data():
    data = ld_mnist()
    images_from_ds=[]
    labels_from_ds=[]
    for z in data.test.take(1):
        images_from_ds=z[0] 
        labels_from_ds=z[1]
    return images_from_ds, labels_from_ds
    


In [162]:

    
print("accuracy")
print(accuracy(fmodel, images_from_ds, labels_from_ds))
print("")

accuracy
0.96875



In [136]:
attacks = [
        fa.FGSM(),
        fa.LinfPGD(),
    ]
epsilons = [
    0.0,
    0.001,
    0.01,
    0.1,
    .2,
    .3,
    .4,
    .5,
    .6,
    .7,
    .8,
    .9,
    1.0,
]

In [137]:
attack_success = np.zeros((len(attacks), len(epsilons), len(images_from_ds)), dtype=np.bool)
for i, attack in enumerate(attacks):
    _, _, success = attack(fmodel, images_from_ds, labels_from_ds, epsilons=epsilons)
    assert success.shape == (len(epsilons), len(images_from_ds))
    success_ = success.numpy()
    assert success_.dtype == np.bool
    attack_success[i] = success_
    print(attack)
    print("  ", 1.0 - success_.mean(axis=-1).round(2))

LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.97 0.97 0.95 0.3  0.04 0.01 0.01 0.01 0.   0.   0.   0.   0.  ]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.97 0.97 0.95 0.15 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]


In [144]:
res1, res2, res3 = fa.FGSM()(fmodel, images_from_ds, labels_from_ds, epsilons=[.1])

In [153]:
res1[0][12]

<tf.Tensor: shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.1       ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.        ],
        [0.1       ],
        [0.1       ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.1       ],
        [0.        ],
        [0.1       ],
        [0.        ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.        ],
        [0.        ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.1       ]],

       [[0.1       ],
        [0.        ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.1       ],
        [0.        ],
        [0.        ],
        [0.1       ],
        [0.1       ],
        [0.        ],
        [0.        ],
        [0.        ],

In [169]:
np.set_printoptions(suppress=True)
res2[0][12]

<tf.Tensor: shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.09999998],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.        ],
        [0.09999998],
        [0.09999998],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.09999998],
        [0.        ],
        [0.09999998],
        [0.        ],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.        ],
        [0.        ],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.09999998]],

       [[0.09999998],
        [0.        ],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.09999998],
        [0.        ],
        [0.        ],
        [0.09999998],
        [0.09999998],
        [0.        ],
        [0.        ],
        [0.        ],

In [170]:
res3[0]

<tf.Tensor: shape=(128,), dtype=bool, numpy=
array([False, False,  True,  True,  True,  True, False,  True,  True,
        True,  True, False,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True, False, False, False,  True,
       False,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
       False,  True, False, False,  True, False,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True, False,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True, False,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True, False,  True,  True,
       False, False,  True,  True, False,  Tr

# Helper functions

In [159]:
def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 0) or (index == 2) or (index == 4):
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index+1].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            flat_weights_to_prune_df = pd.DataFrame(flat_weights_to_prune)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            #print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+1].shape
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            print('yoyoyo',type(weights_to_prune[index+1]))
            weights_to_prune[index+1] = weights_reshaped
    #print(weights_to_prune)
    return weights_to_prune

In [158]:
def prune(model, pruning_ratio):
    pruned_weights = prune_weights(model, pruning_ratio)
    model.set_weights(pruned_weights)
    return model
    

In [8]:
def show_batch(image_batch,label_batch):
    plt.figure(figsize=(10,10))
    for n in range(10):
        img = tf.reshape(image_batch[n], (28,28))
        ax = plt.subplot(5,5,n+1)
        plt.imshow(img)
        plt.title(np.argmax(label_batch[n]))
        plt.axis('off')
show_batch( x_fgm, adv_preds)
show_batch(clean_inputs,clean_preds)

NameError: name 'x_fgm' is not defined

In [ ]:
#dont need this now hopefully - just for debugging reasons
def accuracy(fmodel, inputs, labels):
    inputs_, labels_ = ep.astensors(inputs, labels)
    del inputs, labels

    predictions = fmodel(inputs_).argmax(axis=-1)

    accuracy = (predictions == labels_).float32().mean()
    return accuracy.item()

loss: 1.4662 - accuracy: 0.9951 - val_loss: 1.4805 - val_accuracy: 0.9808
LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.98 0.97 0.97 0.27 0.09 0.02 0.   0.   0.   0.   0.   0.   0.  ]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.98 0.97 0.97 0.16 0.02 0.   0.   0.   0.   0.   0.   0.   0.  ]
   
   
   
loss: 1.4653 - accuracy: 0.9959 - val_loss: 1.4795 - val_accuracy: 0.9814
LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.98 0.98 0.98 0.33 0.06 0.03 0.   0.   0.   0.   0.   0.   0.  ]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.98 0.98 0.98 0.16 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]



loss: 1.4648 - accuracy: 0.9965 - val_loss: 1.4794 - val_accuracy: 0.9821
LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.98 0.98 0.97 0.36 0.06 0.01 0.01 0.   0.   0.   0.   0.   0.  ]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.98 0.98 0.97 0.13 0.01 0.   0.   0.   0.   0.   0.   0.   0.  ]



loss: 1.4640 - accuracy: 0.9972 - val_loss: 1.4793 - val_accuracy: 0.9822
LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.98 0.98 0.98 0.4  0.07 0.02 0.01 0.   0.   0.   0.   0.   0.  ]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.98 0.98 0.98 0.11 0.01 0.   0.   0.   0.   0.   0.   0.   0.  ]



loss: 1.4665 - accuracy: 0.9958 - val_loss: 1.4847 - val_accuracy: 0.9765
LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.98 0.98 0.95 0.3  0.1  0.02 0.02 0.02 0.01 0.01 0.02 0.01 0.01]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.98 0.98 0.95 0.1  0.03 0.01 0.   0.   0.   0.   0.   0.   0.  ]



loss: 2.1755 - accuracy: 0.3708 - val_loss: 2.1757 - val_accuracy: 0.3740
LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False)
   [0.31 0.28 0.28 0.22 0.18 0.16 0.15 0.12 0.12 0.1  0.1  0.1  0.09]
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True)
   [0.31 0.22 0.22 0.13 0.05 0.02 0.02 0.01 0.03 0.01 0.02 0.01 0.01]